In [ ]:
from qiskit import IBMQ
IBMQ.save_account('PRIVATE',
                     overwrite=True)
provider = IBMQ.load_account()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np

from qiskit import BasicAer
from qiskit.tools.visualization import plot_histogram
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance

from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator
from collections import OrderedDict
import math

In [ ]:
def get_knapsack_operator(values, weights, max_weight):

    y_size = int(math.log(max_weight, 2)) + 1
    n = len(values)
    num_values = n + y_size
    pauli_list = []
    shift = 0

    for i in range(n):
        coefficient = weights[i] * values[i]

        xp = np.zeros(num_values, dtype=np.bool)
        zp = np.zeros(num_values, dtype=np.bool)
        zp[i] = not zp[i]
        pauli_list.append([coefficient, Pauli(zp, xp)])
        shift -= coefficient

    return WeightedPauliOperator(paulis=pauli_list), shift

In [ ]:
values = [120, 54, 28, 49]
weights = [13, 6, 15, 9]
w_max = 32

qubitOp, offset = get_knapsack_operator(values, weights, w_max)

In [ ]:
from qiskit.circuit.library import TwoLocal

seed = 10000

spsa = SPSA(max_trials=100)
ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')
vqe = VQE(qubitOp, ansatz, spsa)

backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend, shots=100, seed_simulator=seed, seed_transpiler=seed)

result_shots = vqe.run(quantum_instance)

most_lightly_shots = result_shots['eigvecs'][0]
t = []
for k, v in most_lightly_shots.items():
    t.append((k,v))

In [ ]:
maxcount=0
for i in range(len(t)):
    a = t[i][1]
    if maxcount<a:
        maxcount=a
        
for i in range(len(t)):
    if t[i][1] == maxcount:
        shot=t[i][0]
        
print(shot[:len(values)])

In [ ]:
ansatz.draw('mpl', scale=0.5)

In [ ]:
import networkx as nx
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile, assemble
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram

In [ ]:
n = 4
V = [120, 54, 28, 49]
E = [(120,13,1.0),(54,6,1.0),(28,15,1.0),(49,9,1.0)] 

G = nx.Graph()
G.add_nodes_from(V)
G.add_weighted_edges_from(E)

default_axes = plt.axes(frameon=True)
pos          = nx.spring_layout(G)

nx.draw_networkx(G, node_color=colors, node_size=600, alpha=1, ax=default_axes, pos=pos)

In [ ]:
step_size   = 0.1;

a_gamma         = np.arange(0, np.pi, step_size)
a_beta          = np.arange(0, np.pi, step_size)
a_gamma, a_beta = np.meshgrid(a_gamma,a_beta)

F1 = 3-(np.sin(2*a_beta)**2*np.sin(2*a_gamma)**2-0.5*np.sin(4*a_beta)
        *np.sin(4*a_gamma))*(1+np.cos(4*a_gamma)**2)

result = np.where(F1 == np.amax(F1))
a      = list(zip(result[0],result[1]))[0]

gamma  = a[0]*step_size;
beta   = a[1]*step_size;

In [ ]:
QAOA = QuantumCircuit(len(V), len(V))

QAOA.h(range(len(V)))

for edge in E:
    k = edge[0]
    l = edge[1]
    
QAOA.rx(2*beta, range(len(V)))
QAOA.measure(range(len(V)),range(len(V)))

QAOA.draw('mpl')

In [ ]:
shots = 1000

TQAOA = transpile(QAOA, backend)
qobj = assemble(TQAOA, shots=shots)
QAOA_results = backend.run(qobj).result()

plot_histogram(QAOA_results.get_counts(),figsize = (8,6),bar_labels = False)
print(QAOA_results.get_counts())

In [ ]:
import operator
circ={}
circ=QAOA_results.get_counts()

a=max(circ.items(), key=operator.itemgetter(1))[0]
b=max(circ.items(), key=operator.itemgetter(1))[1]
print(a)
print(b)